## EnumOutputParser
**미리 정의된 선택지 중에서만 답변**을 받고 싶을 때 사용하는 파서입니다.

**주요 특징:**

- **제한된 선택지 강제**: 정의된 옵션 외의 답변 방지
- **분류 작업에 최적**: 카테고리 분류에 특화
- **일관된 응답 보장**: 항상 예측 가능한 결과
- **다국어 지원**: Enum 값에 한국어 등 다양한 언어 사용 가능

**활용 사례:**

- 감정 분석 (긍정/부정/중립)
- 우선순위 분류 (높음/보통/낮음)
- 카테고리 분류
- 품질 등급 평가

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# OutputFixingParser
# -> 다른 파서가 실패했을 때 자동으로 수정을 시도하는 메타 파서
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint
import os

In [2]:
# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

# EnumOutputParser 초기화
parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

# 프롬프트 템플릿
template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)   

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

print("모델 및 파서 설정 완료")

# 테스트 텍스트
texts = [
    "이 제품 정말 좋아요! 완전 만족스러워요.",
    "서비스가 너무 느리고 불친절했습니다.",
    "오늘은 비가 온다네요.",
    "배송은 빠르지만 품질이 아쉽습니다.",
    "최고의 경험이었습니다!",
    "완전 실망했어요... 최악이에요"
]

print(f"테스트할 텍스트 {len(texts)}개 준비 완료")

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 중립
모델 및 파서 설정 완료
테스트할 텍스트 6개 준비 완료


In [3]:
# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

=== 실제 감정 분석 결과 ===

1. 텍스트: 이 제품 정말 좋아요! 완전 만족스러워요.
   감정: 긍정 

2. 텍스트: 서비스가 너무 느리고 불친절했습니다.
   감정: 부정 

3. 텍스트: 오늘은 비가 온다네요.
   감정: 중립 

4. 텍스트: 배송은 빠르지만 품질이 아쉽습니다.
   감정: 부정 

5. 텍스트: 최고의 경험이었습니다!
   감정: 긍정 

6. 텍스트: 완전 실망했어요... 최악이에요
   감정: 부정 

=== 결과 요약 ===
성공: 6/6 (100.0%)
실패: 0/6
